In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np

In [3]:
songs = pd.read_csv("songs.csv")

In [5]:
songs.head()

,track,artist
0,"""Yuck!"" (featuring Lil Wayne )",2 Chainz
1,"""Crack""",2 Chainz
2,"""Dope Peddler""",2 Chainz
3,""" No Lie "" (featuring Drake )",2 Chainz
4,""" Birthday Song "" (featuring Kanye West )",2 Chainz


In [47]:
def clean_track(track):
    s = track
    s = re.sub(r'\(([^\)]+)\)','',s)
    
    s= re.sub(r'\"','',s )
    s = re.sub(r'[!?]','',s)
    s = re.sub(r'[./]+',' ',s)
    s = s.strip()
    s = s.replace(" ","-")
    s=s.lower()
    return s

In [48]:
songs["genius_track"] = songs["track"].apply(clean_track)

In [55]:
def genius_artist(name):
    name =name[0]+name[1:].lower()
    name= name.replace(" ","-")
    return name

In [56]:
genius_artist("2 Chainz")

'2-chainz'

In [58]:
songs["genius_artist"] = songs["artist"].apply(genius_artist)

In [134]:
def get_lyrics(genius_artist,genius_track):
    base = 'https://genius.com/'
    url = base+genius_artist+"-"+genius_track+"-lyrics"
    page = requests.get(url)
    soup = BeautifulSoup(page.text,'html.parser')
    try:
        p = soup.find_all("p")
        s = p[0].text
        s = re.sub(r'\[.*?\]','',s)
        s = s.split("\n")
        s = [ x for x in s if x != ""]
        return s
    except:
        return "error"

In [135]:
soup = get_lyrics(songs["genius_artist"][0], songs["genius_track"][0])

In [136]:
songs.head()

,track,artist,genius_track,genius_artist
0,"""Yuck!"" (featuring Lil Wayne )",2 Chainz,yuck,2-chainz
1,"""Crack""",2 Chainz,crack,2-chainz
2,"""Dope Peddler""",2 Chainz,dope-peddler,2-chainz
3,""" No Lie "" (featuring Drake )",2 Chainz,no-lie,2-chainz
4,""" Birthday Song "" (featuring Kanye West )",2 Chainz,birthday-song,2-chainz


In [137]:
test= songs[0:6000]


In [139]:
test["lyrics"] = test.apply(lambda x : get_lyrics(x["genius_artist"], x["genius_track"]), axis=1)

C:\Users\cwcol\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [141]:
test.to_csv("test_songs.csv", index=False)

In [144]:
type(test["lyrics"][0])

list

In [146]:
import pickle

with open('test_lyrics', 'wb') as fp:
    pickle.dump(test, fp)